In [ ]:
"""
Random Forest Script for Operation IceBridge Airborne Topographic Mapper L2 Icessn Elevation, Slope, and Roughness, v.2 Data

This script uses a random forest regression to identify the most important predicitve features for the ILATM2 data. 
The predicitve features are climate variables from PROMICE and GC-NET on-ice automated weather stations. 
The ILATM2 is filtered (FilterILATM2byAWS.ipynb) to be within 5 km of these stations. 
Climate variable metrics are calculate for each week from the month preceeding the ATM flyover dates at each station. 
An elevation of 1500 m is used to delineate stations above/below the equilibrium line altitude. 
"""

In [1]:
# Import necessary libraries 

import math
import numpy as np
import pandas as pd
import seaborn as sns
import pypromice as pp
from scipy import stats
import pypromice.get as pget
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV, cross_val_score, RandomizedSearchCV, train_test_split
from sklearn.feature_selection import RFE, SelectFromModel
import geopandas as gpd
from matplotlib.colors import LogNorm
from sklearn.linear_model import LinearRegression

In [ ]:
stations = {
    "cen1": ("cen1_day.csv", "./CEN/CEN.csv"),
    "cp1": ("cp1_day.csv", "./Crawford Point/Crawford Point.csv"),
    "dy2": ("dy2_day.csv", "./DYE-II/DYE-II.csv"),
    "egp": ("egp_day.csv", "./EGP/EGP.csv"),
    "hum": ("hum_day.csv", "./Humboldt/Humboldt.csv"),
    "jar": ("jar_day.csv", "./JAR/JAR.csv"),
    "kan_b": ("kan_b_day.csv", "./KAN_B/KAN_B.csv"),
    "kan_l": ("kan_l_day.csv", "./KAN_L/KAN_L.csv"),
    "kan_m": ("kan_m_day.csv", "./KAN_M/KAN_M.csv"),
    "kan_u": ("kan_u_day.csv", "./KAN_U/KAN_U.csv"),
    "kpc_l": ("kpc_l_day.csv", "./KPC_L/KPC_L.csv"),
    "kpc_u": ("kpc_u_day.csv", "./KPC_U/KPC_U.csv"),
    "nem": ("nem_day.csv", "./NEEM/NEEM.csv"),
    "nse": ("nse_day.csv", "./NASA-SE/NASA-SE.csv"),
    "nuk_l": ("nuk_l_day.csv", "./NUK_L/NUK_L.csv"),
    "nuk_n": ("nuk_n_day.csv", "./NUK_N/NUK_N.csv"),
    "nuk_u": ("nuk_u_day.csv", "./NUK_U/NUK_U.csv"),
    "qas_l": ("qas_l_day.csv", "./QAS_L/QAS_L.csv"),
    "qas_m": ("qas_m_day.csv", "./QAS_M/QAS_M.csv"),
    "qas_u": ("qas_u_day.csv", "./QAS_U/QAS_U.csv"),
    "sdl": ("sdl_day.csv", "./Saddle/Saddle.csv"),
    "sco_l": ("sco_l_day.csv", "./SCO_L/SCO_L.csv"),
    "sco_u": ("sco_u_day.csv", "./SCO_U/SCO_U.csv"),
    "swc": ("swc_day.csv", "./Swiss Camp/Swiss Camp.csv"),
    "tas_u": ("tas_u_day.csv", "./TAS_U/TAS_U.csv"),
    "thu_l": ("thu_l_day.csv", "./THU_L/THU_L.csv"),
    "thu_u": ("thu_u_day.csv", "./THU_U/THU_U.csv"),
    "thu_u2": ("thu_u2_day.csv", "./THU_U2/THU_U2.csv"),
    "tun": ("tun_day.csv", "./Tunu-N/Tunu-N.csv"),
    "upe_u": ("upe_u_day.csv", "/UPE_U/UPE_U.csv")
}

In [3]:
def load_station_data(station_name, station_file): # Function to load AWS + ILATM2 data for each station 
    aws_data_file, atm_file = station_file  # Unpack file paths for AWS (weather station) data and ATM (surface roughness) data
    aws_data = pget.aws_data(aws_data_file) # Retrieve AWS data using the pypromice API
    aws_data.index = pd.to_datetime(aws_data.index) # Convert the AWS data index to datetime format
    atm_data = pd.read_csv(atm_file) #  Load ILATM2 data within the 5 km radius 
    atm_data['Date'] = pd.to_datetime(atm_data[['Year', 'Month', 'Day']]) # Create a new 'Date' column 
    return aws_data, atm_data

In [ ]:
station_data = {} # Empty dictionary to store data for each station
for station, file in stations.items(): # Loop over each station in the stations dictionary 
    aws_data, atm_data = load_station_data(station, file) # Load AWS and ATM data for the current station using the 'load_station_data' function
    station_data[station] = {'aws': aws_data, 'atm': atm_data} # Store the loaded AWS and ATM data for stations in the 'station_data' dictionary; the key is the station name, and the value is a dictionary containing 'aws' and 'atm' DataFrames

In [ ]:
day_offsets = [(0, 7), (7, 14), (14, 21)] # Ranges of days prior to the ILATM2 data collection date; (start_day, end_day) pairs
variables = ['p_u', 't_u', 'rh_u_cor', 'qh_u', 'wspd_u','dsr_cor', 'cc', 'dlhf_u', 'dshf_u','wdir_u', 'albedo', 't_surf', 'z_boom_u'] # Variables to download for each station with the pypromice API 

In [7]:
def process_dataset(atm_data, dataset, day_offsets, variables): # Function to compute various ATM+AWS stats for time ranges 
    comp_list = [] # Empty list to store DataFrames for each date

    for date in atm_data['Date'].unique(): # Loop through each unique date in the ATM data
        data_for_date = {'Date': date} # Dictionary to store dates

        for start_offset, end_offset in day_offsets: # Loop over each pair of start and end days from day_offsets
            start_range = date - timedelta(days=end_offset) # Calculate start and end dates for time range based on offsets
            end_range = date - timedelta(days=start_offset)
            filt_df = dataset[(dataset.index >= start_range) & (dataset.index <= end_range)]  # Filter the dataset to include only the rows within the calculated date range
            
            for var in variables: # Loop over each variable in the 'variables' list
                if var in dataset.columns: # If the variable exists in the dataset columns...
                    
                    if var != 'z_boom_u':  # If the variable is not 'z_boom_u', calculate average, max, and min values
                        data_for_date[f'{var}_{end_offset}_avg'] = filt_df[var].mean()
                        data_for_date[f'{var}_{end_offset}_max'] = filt_df[var].max()
                        data_for_date[f'{var}_{end_offset}_min'] = filt_df[var].min()
                    
                    if var == 'z_boom_u':  # If the variable is 'z_boom_u', calculate the difference between max and min
                        data_for_date[f'{var}_{end_offset}_diff'] = filt_df[var].max() - filt_df[var].min()
                
                else: # If the variable does not exist in the dataset, set entry as NaN 
                    data_for_date[f'{var}_{end_offset}_stats'] = float('nan')
 
        comp_list.append(pd.DataFrame([data_for_date])) # Appendstatistics for the current date as a DataFrame to the comp_list
    
    comp_df = pd.concat(comp_list, ignore_index=True) # Concatenate all individual DFs for each date into a single DFs
    rms_df = atm_data.groupby('Date')['RMS_Fit(cm)'].mean().reset_index() # Calculate the mean RMS_Fit and elevation for each date
    height_df = atm_data.groupby('Date')['WGS84_Ellipsoid_Height(m)'].mean().reset_index()

    comp_df = pd.merge(comp_df, rms_df, on='Date', how='left')  # Merge statistics with the RMS_Fit and elevation
    comp_df = pd.merge(comp_df, height_df, on='Date', how='left')

    return comp_df

In [ ]:
all_elev_datasets = {
   'cen': (station_data['cen1']['atm'], station_data['cen1']['aws']),
   'cp': (station_data['cp1']['atm'], station_data['cp1']['aws']),
   'dy2': (station_data['dy2']['atm'], station_data['dy2']['aws']),
   'egp': (station_data['egp']['atm'], station_data['egp']['aws']),
   'hum': (station_data['hum']['atm'], station_data['hum']['aws']),
   'jar': (station_data['jar']['atm'], station_data['jar']['aws']),
   'kan_b': (station_data['kan_b']['atm'], station_data['kan_b']['aws']),
   'kan_l': (station_data['kan_l']['atm'], station_data['kan_l']['aws']),
   'kan_m': (station_data['kan_m']['atm'], station_data['kan_m']['aws']),
   'kan_u': (station_data['kan_u']['atm'], station_data['kan_u']['aws']),
   'kpc_l': (station_data['kpc_l']['atm'], station_data['kpc_l']['aws']),
   'kpc_u': (station_data['kpc_u']['atm'], station_data['kpc_u']['aws']),
   'nem': (station_data['nem']['atm'], station_data['nem']['aws']),
   'nse': (station_data['nse']['atm'], station_data['nse']['aws']),
   'nuk_l': (station_data['nuk_l']['atm'], station_data['nuk_l']['aws']),
   'nuk_n': (station_data['nuk_n']['atm'], station_data['nuk_n']['aws']),
   'nuk_u': (station_data['nuk_u']['atm'], station_data['nuk_u']['aws']),
   'qas_l': (station_data['qas_l']['atm'], station_data['qas_l']['aws']),
   'qas_m': (station_data['qas_m']['atm'], station_data['qas_m']['aws']),
   'qas_u': (station_data['qas_u']['atm'], station_data['qas_u']['aws']),
   'sdl': (station_data['sdl']['atm'], station_data['sdl']['aws']),
   'sco_l': (station_data['sco_l']['atm'], station_data['sco_l']['aws']),
   'sco_u': (station_data['sco_u']['atm'], station_data['sco_u']['aws']),
   'swc': (station_data['swc']['atm'], station_data['swc']['aws']),
   'tas_u': (station_data['tas_u']['atm'], station_data['tas_u']['aws']),
   'thu_l': (station_data['thu_l']['atm'], station_data['thu_l']['aws']),
   'thu_u': (station_data['thu_u']['atm'], station_data['thu_u']['aws']),
   'thu_u2': (station_data['thu_u2']['atm'], station_data['thu_u2']['aws']),
   'tun': (station_data['tun']['atm'], station_data['tun']['aws']),
   'upe_u': (station_data['upe_u']['atm'], station_data['upe_u']['aws']),
}

In [ ]:
extracted_data = pd.DataFrame() # Initialize DF for extracted data 
results = {} # Initialize dictionary to store results 

for name, (atm_data, dataset) in all_elev_datasets.items(): # Loop through datasets in 'all_elev_datasets'
    results[name] = process_dataset(atm_data, dataset, day_offsets, variables) # Process datasets with'process_dataset' function and store results in 'results' dict

extracted_data = pd.concat(results.values(), ignore_index=True) # Concatenate all the DFs in the 'results' dictionary 
extracted_data = extracted_data.dropna(axis=1, how='all')
extracted_data = extracted_data.dropna()
extracted_data = extracted_data.drop(columns='Date')

total = extracted_data # Store cleaned DF
above = extracted_data[extracted_data['Elevation [m]'] >= 1500] # Filter data for above/below ELA 
below = extracted_data[extracted_data['Elevation [m]'] < 1500]

In [ ]:
def plot_spearman_correlations(extracted_data, target_variable, label): # Function to plot Spearman correlation coefficients for target variable
    spearman_corr = extracted_data.corr(method='spearman')[target_variable].drop(target_variable).sort_values(ascending=False) # Compute Spearman's for each variable + target variable 
    low_corr_vars = spearman_corr[abs(spearman_corr) < 0.3] # Identify variables with a correlation < |0.3|
    print(low_corr_vars.index.tolist()) # Print list of low correlation variables 

    plt.figure(figsize=(20, 10))
    sns.barplot(x=spearman_corr.index, y=spearman_corr.values, palette='coolwarm') # Barplot for vizualization of coefficients 
    plt.title(f'Spearman Correlation Coefficients with Surface Roughness{label}', fontsize=20)
    plt.xlabel('Variables', fontsize=20)
    plt.ylabel('Spearman Correlation Coefficient', fontsize=20)
    plt.xticks(rotation=90)
    plt.axhline(-0.3, color='black', linestyle='--')
    plt.axhline(0.3, color='black', linestyle='--')
    plt.tight_layout()
    plt.show()

# Call the function for the 3 groups 
plot_spearman_correlations(total, 'Surface Roughness [cm]', '(All Elevations)')
plot_spearman_correlations(above, 'Surface Roughness [cm]', '(> 1500 m)')
plot_spearman_correlations(below, 'Surface Roughness [cm]', '(< 1500 m)')


In [ ]:
def perform_rf_analysis(data, label, threshold="mean"): # Function to perform Random Forest regression analysis
   feature_columns = [col for col in data.columns if col not in ['Surface Roughness [cm]', 'Date']] # Select feature columns (exclude target variable and date)
   
   X = data[feature_columns] # Split data into features (X) and target variable (y)
   y = data['Surface Roughness [cm]']

   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Split the data into training and testing sets (80% train, 20% test)
   
   rf = RandomForestRegressor(n_estimators=100, random_state=42) # Initialize Random Forest model with 100 estimators and a fixed random state for reproducibility
   rf.fit(X_train, y_train) # Train the model on the training data

   selector = SelectFromModel(rf, threshold=threshold, prefit=True) # Use SelectFromModel to select features based on the importance calculated by the Random Forest
   
   X_selected_train = selector.transform(X_train) # Transform both training and test data to only include the selected features
   X_selected_test = selector.transform(X_test)
   selected_features = np.array(feature_columns)[selector.get_support()] # Extract the selected feature names

   rf_selected = RandomForestRegressor(n_estimators=100, random_state=42) # Initialize a new Random Forest model for the selected features
   rf_selected.fit(X_selected_train, y_train) # Retrain the model using only the selected features

   param_grid = { # Set up a parameter grid for hyperparameter tuning
       'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20, 30],
       'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4],
       'bootstrap': [True, False]
   }

   grid_search = GridSearchCV(estimator=rf_selected, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error') # Perform GridSearchCV to find the best model using cross-validation and the specified parameter grid
   grid_search.fit(X_selected_train, y_train) # Fit the grid search on the training data
   best_rf = grid_search.best_estimator_  # Get the best model from the grid search
   best_rf.fit(X_selected_train, y_train) # Retrain the best model
   y_pred = best_rf.predict(X_selected_test) # Predict on the test data

   mae = mean_absolute_error(y_test, y_pred) # Calculate performance metrics: MAE, RMSE, and R²
   rmse = np.sqrt(mean_squared_error(y_test, y_pred))
   r2 = r2_score(y_test, y_pred)
   print(f'{label} - MAE: {mae}, RMSE: {rmse}, R2: {r2}')

   feature_importances = best_rf.feature_importances_ # Get the feature importances from the best model
   importance_df = pd.DataFrame({'Feature': selected_features, 'Importance':feature_importances})  # Create a DataFrame to display the features and their importances
   importance_df = importance_df.sort_values(by='Importance',ascending=False)  # Sort the DataFrame by feature importance in descending order

   plt.figure(figsize=(10, 5)) # Plot the feature importances as a bar plot
   plt.bar(importance_df['Feature'], importance_df['Importance'],color='skyblue')
   plt.xlabel('Features', fontsize=12)
   plt.ylabel('Importance', fontsize=12)
   plt.title(f'Feature Importance in Predicting Surface Roughness {label}',fontsize=12)
   plt.xticks(rotation=90)
   plt.show()

   lin_reg = LinearRegression() # Fit a Linear Regression model to the predicted vs. actual values
   y_test_reshaped = np.array(y_test).reshape(-1, 1) # Reshape the test and predicted values for fitting the linear regression model
   y_pred_reshaped = np.array(y_pred).reshape(-1, 1)
   lin_reg.fit(y_test_reshaped, y_pred_reshaped) # Fit the linear regression model
   line = lin_reg.predict(y_test_reshaped) # Predict using the linear regression model

   plt.figure(figsize=(6,4)) # Plot predicted vs actual values with a linear regression line
   plt.scatter(y_test, y_pred, c='blue', marker='o')
   plt.plot(y_test, line, color='red', lw=2)
   plt.xlabel('Actual Surface Roughness [cm]', fontsize=10)
   plt.ylabel('Predicted Surface Roughness [cm]', fontsize=10)
   plt.title(f'Random Forest Predicted vs Actual {label}', fontsize=12)
   plt.text(0.05, 0.95, f'R²: {r2:.2f}', transform=plt.gca().transAxes,fontsize=12, verticalalignment='top', horizontalalignment='left')
   plt.tight_layout()
   plt.show()

   residuals = y_test - y_pred # Calculate residuals
   plt.figure() # Plot the distribution of residuals
   sns.histplot(residuals, kde=True)
   plt.xlabel('Residuals', fontsize=12)
   plt.ylabel('Frequency', fontsize=12)
   plt.title(f'Random Forest Residuals Distribution {label}', fontsize=12)
   plt.show()

# Call the function for the 3 groups 
perform_rf_analysis(total, "(All Elevations)")
perform_rf_analysis(above, "(> 1500 m)")
perform_rf_analysis(below, "(< 1500 m)")
